In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
import pymongo
from splinter import Browser
import requests
import pandas as pd
import lxml.html as lh
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/katiehartmann/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


## Mars News

In [3]:
# read HTML from website
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
# Create a Beautiful Soup object
html = browser.html
soup = bs(html, 'html.parser')
# soup

In [5]:
# Retrieve latest news title and paragraph
title = soup.find_all('div', class_="content_title")[1].text
title

"NASA's Perseverance Mars Rover Mission Honors Navajo Language"

In [6]:
paragraph = soup.find_all('div', class_="article_teaser_body")[0].text
paragraph

'Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.'

## JPL Mars Space Images - Featured Image

In [7]:
# read html from website
jpl_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
featured_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
browser.visit(jpl_image_url)

In [8]:
# Create a Beautiful Soup object
html = browser.html
soup = bs(html, 'html.parser')

In [9]:
# retrieve image url
image_url = soup.find_all('img')[1]['src']
featured_image_url = featured_url + image_url
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


## Mars Facts

In [10]:
# read html from website
facts_url = "https://space-facts.com/mars/"
# browser.visit(facts_url)

# create a Beautfiul Soup object
# html = browser.html
# soup = bs(html, 'html.parser')

In [11]:
tables = pd.read_html(facts_url)
mars_facts_df = tables[0]
mars_facts_df.columns = ['Description', 'Value']
mars_facts_df.set_index('Description', inplace=True)
mars_facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [12]:
fact_table = mars_facts_df.to_html()
fact_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [13]:
fact_table.replace('\n','')
print(fact_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


## Mars Hemispheres

In [14]:
# scrape for images and titles
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
usgs_url = "https://astrogeology.usgs.gov"
browser.visit(hemisphere_url)
html = browser.html
soup=bs(html,'html.parser')

In [15]:
# Extract hemispheres item elements
mars_hems=soup.find('div',class_='collapsible results')
mars_item=mars_hems.find_all('div',class_='item')
hemisphere_image_urls=[]

In [17]:


# loop through hemisphere images
for item in mars_item:
    # Error handling
    try:
        # Extract title
        hem=item.find('div',class_='description')
        title=hem.h3.text
        print(title)
        # Extract image url
        hem_url=hem.a['href']
        browser.visit(usgs_url+hem_url)
        html=browser.html
        soup=bs(html,'html.parser')
        image_src=soup.find('li').a['href']
        print(image_src)
#         if (title and image_src):
#             # Print results
#             print('-'*50)
#             print(title)
#             print(image_src)
        # Create dictionary for title and url
        hem_dict={
            'title':title,
            'image_url':image_src
        }
        hemisphere_image_urls.append(hem_dict)
    except Exception as e:
        print(e)

Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
